# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
# Write your answer here

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

### Libraries used

Glob library is used in order to leverage regular expression capabilities when reading multiple data with similar name.

In [ ]:
import pandas as pd
import numpy as np
import glob, os

### Importing the spreadsheets into a single `DataFrame`

Individual `DataFrames` are read by matching the expression of the file name (*MID#*) and are read into list of dataframes. They are concatenated into a single dataframe `MIDs`.

No header exists in the dataset, so the data is read with appropriate function parameters. In order to be able to track the provenance of the data for the metadata, an additional column *Barcode* is added.

In [ ]:
dataframes = []

# Iterating through all the files satisfying the "regex"
for file in glob.glob(pathname=DATA_FOLDER+'/microbiome/MID*.xls'):
    
    # Reading the file
    dataframe = pd.read_excel(file, 'Sheet 1', header=None)
    
    # Defining the barcode attribute so we can track back rows to metadata
    dataframe['BARCODE']=os.path.basename(file).split(".")[0]

    # Giving semantics to columns
    dataframe.columns = ['Taxon','Count', 'BARCODE']
    dataframe.index.name = 'No.'
    
    # Appending the read dataframe to the list for later concatenation
    dataframes.append(dataframe)

# Concatenating dataframes from the list into single dataframe
MIDs = pd.concat(dataframes, axis=0)

In [ ]:
MIDs.head()

### Combining the DataFrame with metadata

Reading the metadata, with *BARCODE* as a natural choice for the key, as it is supposed to be unique.

In [ ]:
metadata = pd.read_excel(DATA_FOLDER+'/microbiome/metadata.xls', index_col=0)

In [ ]:
metadata

Executing left join on MIDs DataFrame to make a connection with metadata DataFrame. The join is performed on the *BARCODE* column.

In [ ]:
finalDF = MIDs.join(metadata, on='BARCODE')
finalDF.head()

### Index uniqueness

Let us assert if the existing choice to index on the occurence when reading the separate data file results in an unique index:

In [ ]:
finalDF.index.is_unique

It could have been guessed that for uniqueness number of row of each file would not be appropriate, since we have concatenated several such files. Another strategy would be to add additional information unique to the each file, such as *BARCODE*.

In [ ]:
finalDF_unique = finalDF.copy()
finalDF_unique.index = finalDF.index.astype(str) + ' ' + finalDF.BARCODE
finalDF_unique.index.name = 'id'

finalDF_unique.index.is_unique

In [ ]:
finalDF_unique.head()

#### Remark on index uniqueness

Different approaches could be taken in unique index selection. One of the options could have been to use *Taxon* as a primary index. It would still result in a non-unique index when different files are combined in a single DataFrame since same taxons are present in different measurements. On the same note, concatenating the *BARCODE* value would result in an unique index.k

With current approach the row identifier length is kept small (6 characters), while with the aforementioned option such size would both vary and be significantly larger, potentially impacting performance.

In [ ]:
finalDF_unique_h1 = finalDF.copy()
finalDF_unique_h1 = finalDF_unique_h1.set_index(["Taxon","BARCODE"])

finalDF_unique_h1.head()

In [ ]:
finalDF_unique_h1.index.is_unique

Another option which would bring more semantic and group the data would be to perform hierarchical indexing, based on *BARCODE* as level 0, and *Taxon* as level 1 index:

In [ ]:
finalDF_unique_h2 = finalDF.copy()
finalDF_unique_h2 = finalDF_unique_h2.set_index(["BARCODE","Taxon"])

finalDF_unique_h2

In [ ]:
finalDF_unique_h2.index.is_unique

### Finally, handling NaN values

For this purpose we will use `finalDF_unique` DataFrame, but the logic remains the same whichever type of indexing we pick.

In [ ]:
finalDF_unique.fillna('unknown', inplace=True)

finalDF_unique.head()

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here